In [1]:
import draft

# users = draft.get_most_location_id(10)

# get top users
# users = draft.get_top_n_user(20)

In [2]:
import draft
'''
[{'key': 256478435, 'value': 2464}, 
{'key': 931566656115032000, 'value': 1211}, 
{'key': 3958911379, 'value': 1093}, 
{'key': 979984755125928000, 'value': 958}, 
{'key': 133042870, 'value': 871},  # selected user
{'key': 971914328982011900, 'value': 823}, 
{'key': 57314631, 'value': 746}, 
{'key': 2427740995, 'value': 717}, 
{'key': 378439995, 'value': 686}, 
{'key': 347883924, 'value': 655}] # candidate user
'''

'''
candidate tweets
{'key': 3635899574, 'value': 86}
{'key': 1479269839, 'value': 75}
'''



# get user with coordinate time and tweets
# user1 = draft.get_tweet_location_coordinate(3635899574)
# user2 = draft.get_tweet_location_coordinate(1479269839) # selected user -> user2
user2 = draft.get_user_all_tweet(1479269839)
for i in user2[:10]:
    print(i)

# user_tweets_1 = getTweets(user1)
# user_tweets_2 = getTweets(user2)

# print('-----user1-----')
# user1_tweets = []
# for i in user1:
#     print(i[0])
# print('-----user2-----')
# for i in user2:
#     print(i[1])
# get user tweets
# user_tweets_1 = draft.get_user_tweets(133042870)
# user_tweets_2 = draft.get_user_tweets(347883924)
# print(len(user_tweets_1))
# print(len(user_tweets_2))



['RT @olivercrocker: Great pulling power from the mighty Eric Richard (Sgt. Bob Cryer) so far on this #BankHolidayWeekend #TheBill legend has…', datetime.datetime(2018, 5, 7, 19, 51, 38), None]
['Just posted a photo @ Pie In The Sky Roadhouse https://t.co/RYHsdnw0pq', datetime.datetime(2018, 5, 7, 5, 5, 33), {'type': 'Point', 'coordinates': [150.58197069, -33.51530863]}]
['Allendale @ Goodnight https://t.co/wPsNNdQBwP', datetime.datetime(2018, 5, 7, 4, 52, 3), {'type': 'Point', 'coordinates': [143.332, -34.9123]}]
['RT @Arsenal: #MerciArsène https://t.co/hwab7mG6XL', datetime.datetime(2018, 5, 6, 14, 9, 25), None]
['RT @Youaremygooners: #thankful for the tireless work that he has done for our club. Looking at where we started &amp; where we are today, he ha…', datetime.datetime(2018, 5, 6, 14, 8, 37), None]
['RT @BlogtorWho: Video of the Day – Doctor Who -The Silurians,\xa01970 https://t.co/FRxz75h1Wy https://t.co/n0QB8gLXRp', datetime.datetime(2018, 5, 6, 14, 7, 36), None]
['RT @missy

In [4]:
# number of tweets
print(len(user2))

3202


In [9]:
%%time
# topic modelling----------------
import re
import string
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
import nltk
import gensim
from gensim import corpora

def getTweets(user):
    tweet = []
    for i in user:
        if not i[0].startswith('RT') and not i[0].startswith('Just posted a photo'):
            tweet.append(i[0])
    return tweet

user_tweets_2 = getTweets(user2)

def getTweetsCorpus(user_tweets):
    punctuation = set(string.punctuation)
    stop = set(stopwords.words('english'))
    stop.update(['I', 'I’m', 'us', 'u', 'He’s', 'We', 'And', 'You','When', 'Yeah','That’s', 'It', 'The',
                 'This','Its','Im', 'That', 'Oh', 'one', 'LOL', 'A', 'He','Just'])
    punctuation.add('…')

    lemma = WordNetLemmatizer()
    pattern = re.compile('(#[A-Za-z0-9]+)|(@[A-Za-z0-9]+)')
    
    corpus = []
    for i in user_tweets:
        # remove @ and # 
        tweet = re.sub(pattern,' ', i).strip()
        # remove http...
        tweet = [i for i in tweet.split() if not i.startswith('http')]
    
        # remove verb
        pos_tag = nltk.pos_tag(tweet)
        tweet = [i[0] for i in pos_tag if (not i[1].startswith('V')) and (i[1] != 'IN') ]
    
        # remove stop words
        tweet = " ".join([i for i in tweet if i not in stop])
        # remove puncutation
        tweet = ''.join([i for i in tweet if i not in punctuation])
        # normalise the word
        # tweet = [lemmatize(word) for word in tweet.split()]
        # not lemmatize
        tweet = [word for word in tweet.split()]
    
        corpus.append(tweet)
        # print(tweet)
    return corpus


# user1_corpus = getTweetsCorpus(user_tweets_1)
user2_corpus = getTweetsCorpus(user_tweets_2)

def getTopicsLDA(corpus):

    # Creating the term dictionary of our courpus, where every unique term is assigned an index.
    dictionary = corpora.Dictionary(corpus)

    # filter the extreme tokens
    dictionary.filter_extremes(no_below=2, no_above=0.9)

    # Converting list of corpus into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in corpus]

    # Creating the object for LDA model using gensim library
    Lda = gensim.models.ldamodel.LdaModel

    # Running and Trainign LDA model on the document term matrix.
    ldamodel = Lda(doc_term_matrix, num_topics=7, id2word = dictionary, passes=50, dtype=np.float64)

    return ldamodel

# user1_topics_lda = getTopicsLDA(user1_corpus)
user2_topics_lda = getTopicsLDA(user2_corpus)

# get probability distribution for each tweet
# probability = ldamodel.get_document_topics(doc_term_matrix)
# print(len(probability))
# for i in probability:
#     print(i)



CPU times: user 25.9 s, sys: 115 ms, total: 26 s
Wall time: 26.1 s


In [10]:
# year = user2[0][1].year
# month = user2[0][1].month
# day = user2[0][1].day
# hour = user2[0][1].hour
# minute = user2[0][1].minute
# print(year)
# print(month)
# print(day)
# print(hour)
# print(minute)

In [11]:
def showTopics(user_topics_lda): 
    for i in user_topics_lda.print_topics(num_words=5):
        print(i)
        

#print(showTopics(user1_topics_lda))
print('\n')
print(showTopics(user2_topics_lda))



(0, '0.073*"New" + 0.067*"Wales" + 0.067*"South" + 0.043*"frain" + 0.021*"Australia"')
(1, '0.021*"Cafe" + 0.019*"thing" + 0.016*"always" + 0.016*"Queen" + 0.016*"White"')
(2, '0.159*"📷" + 0.025*"Awesome" + 0.018*"So" + 0.013*"Well" + 0.012*"awesome"')
(3, '0.032*"Retreat" + 0.032*"Wombatz" + 0.019*"one" + 0.015*"show" + 0.014*"Yes"')
(4, '0.048*"awesome" + 0.036*"sl" + 0.022*"work" + 0.018*"My" + 0.016*"OMG"')
(5, '0.055*"📷" + 0.029*"Screen" + 0.024*"caps" + 0.017*"Frain" + 0.016*"True"')
(6, '0.037*"Happy" + 0.028*"📷" + 0.014*"birthday" + 0.014*"Tudors" + 0.013*"James"')
None


In [3]:
topic_key_word = ['New South Wales', 
                  'Boorowa','Cafe',
                  'frain','Star', 'Trek',
                  'Wombatz','Love',
                  'Happy','photo',
                  'James','work','Queen',
                  'OMG','Australia','birthday']

def getTweetsByKeyWord(key_word):
    topics_sentence = {}
    for i in user2:
        if i[0].startswith('Just posted a photo') or i[0].startswith('RT'):
            continue
        time = i[1]
        for j in key_word:
            if j in i[0]:
                temp = topics_sentence.get(j, set())
                temp.add((i[0], time))
                topics_sentence[j] = temp
    return topics_sentence

topic_sentence = getTweetsByKeyWord(topic_key_word)

def getOriginalTweets(uset_topic_lad):
    pattern = re.compile('"')
    topics = {}
    # list of (int, list of (str, float))
    for i in uset_topic_lad.print_topics(num_words=5):
        topic = i[0]
        key_word = i[1].split('+')
        key_word = [re.sub(pattern,'', i.split("*")[1]).strip() for i in key_word]
        topics[topic] = set(key_word)

    topics_sentence = {}
    for i in user_tweets:
        tweet = [word for word in i.split() if not word.startswith('http')]
        tweet = ' '.join(tweet)
        for k,v in topics.items():
            for word in v:
                if word in tweet:
                    temp = topics_sentence.get(k, set())
                    temp.add(tweet)
                    topics_sentence[k] = temp
    return topics_sentence

# user1_topic = getOriginalTweets(user1_topics_lda)
# user2_topic = getOriginalTweets(user2_topics_lda)


In [9]:
f = open('key_word_tweets.txt', 'w')

for i in topic_sentence:
    f.write('{}\n'.format(i))
f.close()

In [4]:
# write result to a file
def outputTopics(topic_sentence, filename):
    f = open(filename,'w')
    for k, v in topic_sentence.items():
        f.write('\n---------- Topic key word: {} ----------\n'.format(k))
        for i in v:
            f.write('\n{}, => {} \n'.format(i[0], i[1]))
    f.close() 
    
# outputTopics(user1_topic, 'user1_topic.txt')
outputTopics(topic_sentence, 'user2_topic_key_word.txt')

In [13]:
# get tweeting time from 00:00 to 23:59 in one day
from datetime import datetime

def getTweetTime(user):
    time = []
    for i in user:
        time.append(i[1])
    return time

tweet_time = getTweetTime(user2)

time_format = '%a %b %d %H:%M:%S %z %Y'
time_zone = []
for i in range(0, 24, 2):
    time_zone.append([i, i+2])

tweet_time_dict = {}
for i in tweet_time:
    # time_object = datetime.strptime(i, time_format)
    hour = i.hour
    minute = i.minute
    time = hour + float(minute / 60)
    for zone in time_zone:
        if zone[0] <= time <= zone[1]:
            tweet_time_dict[(zone[0],zone[1])] = tweet_time_dict.get((zone[0],zone[1]), 0) + 1
            
print(tweet_time_dict)
    

{(4, 6): 265, (14, 16): 56, (8, 10): 366, (20, 22): 318, (12, 14): 334, (6, 8): 352, (22, 24): 570, (0, 2): 342, (2, 4): 251, (10, 12): 279, (18, 20): 80, (16, 18): 6}


In [36]:
# find outlier tweets
outlier_tweets = []
for i in user2:
    year = i[1].year
    month = i[1].month
    day = i[1].day
    hour = i[1].hour
    minute = i[1].minute
    date = str(year) + '/' + str(month) + '/' + str(day) + ' ' + str(hour) + ':' + str(minute)
    time = time = hour + float(minute / 60)
    if 16 <= time <= 18:
        outlier_tweets.append((i[0], date))

print(len(outlier_tweets))
print(outlier_tweets)

# write outlier tweets to a file
f = open('outlier_tweets.txt','w')
for i in outlier_tweets:
    f.write('\n{}\n'.format(i))
f.close() 

6
[('RT @juliebrowse: Monday looooms ... https://t.co/R3oiMLunhN', '2018/4/15 16:15'), ('RT @swear_trek: https://t.co/EsXMXFfUnk', '2018/4/14 16:51'), ('@deedeesadler Same to you lovely lady', '2018/4/14 16:50'), ('RT @ansonmount: This was something close to my reaction upon hearing that I was cast in @startrekcbs     #StarTrek #StarTrekDiscovery https…', '2018/4/13 16:1'), ('An oldie but a goodie https://t.co/xHJIt4DO7S', '2018/1/24 17:22'), ('@ParkerFilmCo Oh. Thought it was the ship from Lost in Space for a moment. Looks awesome', '2018/1/24 17:22')]


In [40]:
# travel hit map
def getTimeAndLocation(user):
    time_location = []
    for i in user:
        if i[2]:
            time_location.append((i[1], i[2]['coordinates']))
    return time_location

user_time_location = getTimeAndLocation(user2)
travel_map = {}
heat_map = {}
for i in user_time_location:
    coordinate = (i[1][1], i[1][0])
    year = i[0].year
    month = i[0].month
    day = i[0].day
    date = str(year) + '/' + str(month) + '/' + str(day)
    # update the hit map
    heat_map[coordinate] = heat_map.get(coordinate, 0) + 1
    # update the travel location
    travel_location_list = travel_map.get(date, set())
    travel_location_list.add(coordinate)
    travel_map[date] = travel_location_list

heat_map = sorted(heat_map.items(), key=lambda x:x[1], reverse=True)

In [42]:
for i in heat_map:
    print('{} => {}\n'.format(i[0], i[1]))

print('-----------')

for k,v in travel_map.items():
    print('{}, {}\n\n'.format(k,v))
    


(-36.38596635, 148.84645611) => 73

(-33.75379065, 151.29704833) => 46

(-33.6167, 150.817) => 40

(-37.0667, 149.9) => 32

(-36.3667, 148.833) => 24

(-36.93110107, 149.87290142) => 21

(-36.9, 149.9) => 20

(-33.61459, 150.81334) => 17

(-36.9, 149.233) => 14

(-34.4333, 148.733) => 14

(-36.4167, 150.05) => 13

(-33.63857815, 150.79984402) => 13

(-33.85678499, 151.21528387) => 11

(-33.7, 150.55) => 11

(-33.8848, 151.226) => 11

(-33.5, 150.38333333) => 10

(-37.17861111, 149.98222222) => 10

(-33.815171, 151.009996) => 10

(-36.2333, 149.133) => 9

(-36.50421, 148.83459) => 9

(-33.86, 151.211) => 9

(-33.63796992, 150.79586677) => 8

(-36.407189, 150.0546115) => 8

(-33.8833, 151.225) => 8

(-36.4167, 148.333) => 8

(-33.87111111, 151.21333333) => 8

(-37.10338, 149.8801) => 7

(-33.608134, 150.7884737) => 7

(-33.6054276, 150.8226541) => 7

(-33.58625505, 150.85934191) => 6

(-33.6, 150.767) => 6

(-36.8333, 149.367) => 6

(-33.8167, 151.0) => 6

(-33.51530863, 150.58197069) =>